Load Dataset CIFAR-10

In [ ]:
from tensorflow.keras.datasets import cifar10

# Load dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print("Train:", x_train.shape, y_train.shape)
print("Test:", x_test.shape, y_test.shape)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
Train: (50000, 32, 32, 3) (50000, 1)
Test: (10000, 32, 32, 3) (10000, 1)


Koneksi ke MongoDB Atlas

In [ ]:
!pip install pymongo --quiet

In [ ]:
!pip install "pymongo[srv]"

In [ ]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

uri =

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [ ]:
db = client["cifar10_db"]
collection = db["images"]

In [ ]:
sample_doc = {"name": "sample_image", "label": "cat"}
collection.insert_one(sample_doc)
print("✅ 1 document berhasil disimpan")

✅ 1 document berhasil disimpan


In [ ]:
for doc in collection.find().limit(3):
    print(doc)

{'_id': ObjectId('68d7d923b08fa2003280d9e5'), 'name': 'sample_image', 'label': 'cat'}


Siapkan Data

In [ ]:
# Load dataset CIFAR-10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Upload sebagian data (contoh 5000 agar tidak overload)
docs = []
for i in range(5000):
    docs.append({
        "image": x_train[i].tolist(),
        "label": int(y_train[i])
    })

collection.insert_many(docs)
print("✅ CIFAR-10 images uploaded to MongoDB Atlas")

/tmp/ipython-input-3856835012.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  "label": int(y_train[i])


✅ CIFAR-10 images uploaded to MongoDB Atlas


Load Data Dari Mongo ke Python

In [ ]:
import numpy as np

cursor = collection.find().limit(5000)
data = list(cursor)

X, y = [], []
for doc in data:
    if "image" in doc and "label" in doc:  # Check if 'image' and 'label' keys exist
        X.append(np.array(doc["image"]))
        y.append(doc["label"])


X = np.array(X)
y = np.array(y)

print("✅ Data loaded from MongoDB:", X.shape, y.shape)

✅ Data loaded from MongoDB: (4999, 32, 32, 3) (4999,)


Bangun Model Deep Learning (CNN untuk CIFAR-10)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

# Preprocessing
X = X.astype("float32") / 255.0
y = to_categorical(y, 10)

# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# CNN Model
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation="relu", input_shape=(32,32,3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3,3), activation="relu"),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(10, activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Training Model

In [ ]:
# Training
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=64,
    validation_data=(X_val, y_val),
)


Epoch 1/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 141ms/step - accuracy: 0.7270 - loss: 0.7718 - val_accuracy: 0.5540 - val_loss: 1.4295
Epoch 2/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 132ms/step - accuracy: 0.7227 - loss: 0.7664 - val_accuracy: 0.5110 - val_loss: 1.7663
Epoch 3/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 124ms/step - accuracy: 0.7200 - loss: 0.7647 - val_accuracy: 0.5210 - val_loss: 1.6383
Epoch 4/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 139ms/step - accuracy: 0.7571 - loss: 0.6716 - val_accuracy: 0.5560 - val_loss: 1.5388
Epoch 5/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 8s 119ms/step - accuracy: 0.7352 - loss: 0.7097 - val_accuracy: 0.5400 - val_loss: 1.8294
Epoch 6/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 9s 139ms/step - accuracy: 0.7439 - loss: 0.6770 - val_accuracy: 0.5620 - val_loss: 1.4526
Epoch 7/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 10s 138ms/step - accuracy: 0.7565 - loss: 0.6686 - val_accuracy: 0.5530 - val_loss: 1.5361
Epoch 8/50
63/63 ━━━━━━━━━━━━━━━━━━━━ 7s 117ms/step - accuracy: 0.7806 - loss: 0.6043 - val_accuracy: 0

Evaluasi

In [ ]:
test_loss, test_acc = model.evaluate(X_val, y_val, verbose=0)
print(f"✅ Test Accuracy: {test_acc:.2f}")

✅ Test Accuracy: 0.59
